In [2]:
# --- Create a RAG chatbot to answer questions based on Chanakya Neeti PDF ---
# important package installations
!pip install langchain openai pypdf faiss-cpu sentence-transformers
!pip install -U langchain-community pypdf
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.3 MB/s eta 0:00:00


In [33]:
# Mount google drive to get the PDF for Chanakya Neeti
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Get PDF document and number of pages in the document

In [7]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer

# --- Configuration ---
PDF_PATH = "/content/drive/MyDrive/Colab Notebooks/AI-ML(new)/ChanakyaNeeti_in_English.pdf"  # Replace with the actual path to your PDF
# Make sure your OpenAI API key is set as an environment variable
# os.environ["OPENAI_API_KEY"] = "sk-..."

# --- 1. Load the Document ---
print("Loading PDF document...")
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
print(f"Loaded {len(documents)} pages.")

Loading PDF document...
Loaded 14 pages.


## Set environment variables to get Hugging Face Token and Open AI API key

In [ ]:
# Environment Variables
import os
import getpass

# -------- Removed keys/tokens due to security reasons --------
# -------- uncomment and use your API key and tokens from Open AI and Hugging Face respectively --------

# key = ""
# if not os.environ.get("OPENAI_API_KEY"):
#   os.environ["OPENAI_API_KEY"] = key

# hf_token = ""
# if not os.environ.get("HF_TOKEN"):
#   os.environ["HF_TOKEN"] = hf_token

## Split the document to number of chunks based on separation (first prioirity: New Paragraph, second: New line, third: word based and fourth: each character

In [9]:
# --- 2. Split the Text ---
print("Splitting document into chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100,separators=["\n\n", "\n", " ", ""] )

texts = []
for doc in documents:
    texts.extend(text_splitter.split_text(doc.page_content))

print(f"Split into {len(texts)} text chunks.")

Splitting document into chunks...
Split into 173 text chunks.


## Install and use ChromaDB for Vector Enbedding Storage

In [11]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 55.9 MB/s eta 0:0

In [12]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load model & tokenizer
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# #

model_name = "all-MiniLM-L6-v2"

# Load sentence-transformers model directly (optional, to check)
embedding_model = SentenceTransformer(model_name)

# Define Chroma embedding function
hf_embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
)

# Function to get embeddings
def embed_texts(texts):
    return embedding_model.encode(texts).tolist()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'

In [14]:
!rm -rf ./docs/chroma  # remove old database files if any

In [15]:
# Init Chroma
chroma_client = chromadb.PersistentClient(path="./docs/chroma")

collection = chroma_client.get_or_create_collection(
    name="my_collection",
    embedding_function=hf_embedder
)

collection.add(
    documents=texts,
    embeddings = embed_texts(texts),
    ids=[f"id_{i}" for i in range(len(texts))]
)


## Create retrieval of documents based on Vector inclination (similarity based) to the query embedding given as user input

In [16]:
def retrieve_documents(query, collection, top_k=3):
    # Step 1: embed the query
    query_embedding = embed_texts(query)
    print(query)

    # Step 2: query Chroma collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )

    docs = results['documents'][0]
    ids = results['ids'][0]
    return docs, ids

### Create a Hugging Face InferenceClient Chat Completion Chatbot to answer any user prompt based on the PDF context
Ensure the max tokens ensure the chatbot gives a proper reply to your query and the number of similarities (top_k) also defines how many similar contexts do you want your LLM to answer based on.

In [31]:
from huggingface_hub import InferenceClient

def generate_answer(query, collection, top_k=3):
    docs, ids = retrieve_documents(query, collection, top_k=top_k)

    # Create a prompt for the LLM
    context = "\n\n".join([f"[{i+1}] {doc}" for i, doc in enumerate(docs)])
    prompt = f"""
                You are an expert assistant. Answer the following question using ONLY the documents provided below.
                Cite each reference in square brackets corresponding to the documents used.

                Documents:
                {context}

                Question:
                {query}

                Answer with citations:
            """

    messages = [{"role": "user", "content": prompt}]

    client = InferenceClient("meta-llama/Meta-Llama-3-8B-Instruct")

    response = client.chat_completion(messages, max_tokens=500)

    answer = "Assistant: \n" + str(response.choices[0].message["content"])
    return answer

In [32]:
query = "What does the book say about discipline?"
answer = generate_answer(query, collection, 5)
print(answer)

What does the book say about discipline?
Assistant: 
The book discusses discipline in various contexts. 

From a passage in [2], it can be inferred that maintaining discipline is essential to achieve certain goals and maintain a peaceful life, as mentioned in the following lines: "What peace can we expect from a rascal friend?...How can renown be gained by instructing an unworthy disciple?" This implies that discipline is necessary for relationships, education, and personal growth.

The book also talks about the importance of learning from animals and their characteristics, specifically the dog and the ass, as mentioned in [3] and [5]. The dog's qualities such as "obedience to the master" and "bravery" [5] and the ass's qualities like being "unmindful of cold and heat" and "always contented" [5] are considered virtues that should be learned for discipline.

Moreover, the book emphasizes the importance of self-control and discipline in personal life, as mentioned in [3]: "As long as you